In [6]:
import yaml
import ruamel.yaml as ryaml
import timeit
import json

if ryaml.__with_libyaml__:
    print("ruamel.yaml is using the C library")
else:
    print("ruamel.yaml is not using the C library")


class YAMLSafeLoader(yaml.SafeLoader):  # pylint: disable=too-many-ancestors
    """Safe YAML loader wtih some specific datatypes."""

    def construct_python_tuple(self, node):
        """Tuple constructor."""
        return tuple(self.construct_sequence(node))


YAMLSafeLoader.add_constructor("tag:yaml.org,2002:python/tuple", YAMLSafeLoader.construct_python_tuple)

yaml2 = ryaml.YAML(typ="safe")

# Add the custom constructor to the constructor registry
yaml2.constructor.add_constructor("tag:yaml.org,2002:python/tuple", lambda loader, node: tuple(loader.construct_sequence(node)))


def test_normal_yaml():
    with open("birdplan.state", "r") as f:
        config = yaml.load(f, YAMLSafeLoader)


def test_ruamel_yaml():
    with open("birdplan.state", "r") as f:
        config = yaml2.load(f)


def test_json():
    with open("birdplan.state.json", "r") as f:
        config = json.load(f)


def test_json_loads():
    with open("birdplan.state.json", "r") as f:
        data = f.read()
    config = json.loads(data)


def test_json_pretty():
    with open("birdplan.state.json.pretty", "r") as f:
        config = json.load(f)


def test_json_pretty_loads():
    with open("birdplan.state.json.pretty", "r") as f:
        data = f.read()
    config = json.loads(data)


print("Normal yaml : ", timeit.timeit("test_normal_yaml()", setup="from __main__ import test_normal_yaml", number=1))
print("Ruamel yaml : ", timeit.timeit("test_ruamel_yaml()", setup="from __main__ import test_ruamel_yaml", number=1))
print("JSON        : ", timeit.timeit("test_json()", setup="from __main__ import test_json", number=100))
print("JSONS       : ", timeit.timeit("test_json_loads()", setup="from __main__ import test_json_loads", number=100))
print("JSON Pretty : ", timeit.timeit("test_json_pretty()", setup="from __main__ import test_json_pretty", number=100))
print("JSONS Pretty: ", timeit.timeit("test_json_pretty_loads()", setup="from __main__ import test_json_pretty_loads", number=100))

ruamel.yaml is using the C library
Normal yaml :  3.6992921410128474
Ruamel yaml :  1.0288359450059943
JSON        :  0.6750548609998077
JSONS       :  0.7842083179857582
JSON Pretty :  1.2286666650034022
JSONS Pretty:  1.2248095969844144
